## **UNet Training and Testing**
Used in this repository for super-resolution image noise detection, but can be trained for other purposes.

**Code:** https://github.com/zhixuhao/unet

**Paper:** Ronneberger O., Fischer P., Brox T. (2015) U-Net: Convolutional Networks for Biomedical Image Segmentation. In: Navab N., Hornegger J., Wells W., Frangi A. (eds) Medical Image Computing and Computer-Assisted Intervention – MICCAI 2015. MICCAI 2015. Lecture Notes in Computer Science, vol 9351. Springer, Cham. https://doi.org/10.1007/978-3-319-24574-4_28

---

**Installation:** Packages require local installation. Tested on:

cuda 10.0 & cudnn 7

python 3.6.9 (or 3.7.4)

tensorflow 1.14-gpu

keras 2.2.4 - 2.2.5

numpy, matplotlib, opencv-python, h5py, tkinter

---

***Manual Install**

The following packages must be installed manually:

Cuda Toolkit 10.0:  https://developer.nvidia.com/cuda-10.0-download-archive

cuDNN v7.6.5 for CUDA 10.0:  https://developer.nvidia.com/rdp/cudnn-archive

Python 3.6.9:  https://www.python.org/downloads/release/python-369/

  OR

Python 3.7.4 (windows installer):  https://www.python.org/downloads/release/python-374/

---

**Begin:**  The first step, is to gather your data into two directories (folders).  One for the original images to be processed and one for data labels.

If you are testing only, i.e. performing segmentation, labels are not required.  If you are conducting training, labels are required.  Annotation files are not required.

---

**Next:**  Connect to local computer.  
- - - 
***Connecting to Local Runtime:**

First time users follow the setup instructions here:
https://research.google.com/colaboratory/local-runtimes.html

Subsequent users can use the following instructions

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

Enter the following into command prompt (CMD):

*jupyter notebook   --NotebookApp.allow_origin='https://colab.research.google.com'   --port=8888   --NotebookApp.port_retries=0 --allow-root --no-browser*

Copy the http address that appears in the **command window**, near the bottom of the current output. (e.g. *http://localhost:8888/?token=a887174a56c905d5421fc88b2086782d53dfa034a7e690d0*) [Note: do NOT copy this address, it's just an example] [Note2: Probably better to use the "localhost:8888" url rather than the "127.0.0.1:8888".]

Connect to Local Runtime by clicking the down arrow next to the "Connect" button in the top right of this window.  Then click "Connect to local runtime".

Paste the copied url into the input line on the popup window. Click Connect.  (Note: if an old address is already in the line, replace it with the new one)

For more info:
see: https://research.google.com/colaboratory/local-runtimes.html

---

**After That:** Adjust values in each section of the code.  Use defaults if unsure. Run each section.

---

**Then:**  Train or Test

---

In [1]:
#@markdown ___
#@markdown ## **I. Install Dependencies**

#@markdown Do you need to install UNet?
UNet = False #@param {type:"boolean"}
#@markdown Path for UNet Install or to Existing UNet Install
UPATH = "C:\\Users\\UPMC\\PythonCode\\unet" #@param {type:"string"}

if UNet == True:
  try:
    !git clone --quiet https://github.com/zhixuhao/unet.git
  except Exception:
    print("Install git and run again, or go to https://github.com/zhixuhao/unet.git and download/install UNet")

#@markdown Do you need to install other dependencies (i.e. tensorflow, keras)
DEP = False #@param {type: "boolean"}

if DEP == True:
  !pip uninstall -y tensorflow
  !pip install tensorflow_gpu==1.14
  !pip install keras==2.2.4

In [ ]:
#@markdown ___
#@markdown ## **II. Import Libraries**

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()


In [16]:
#@markdown ___
#@markdown ## **III. Import Image Data**

#@markdown ### **A. File Directories?**
#@markdown Common path to folder containing images
ROOT_PATH = 'D:\\Images\\train' #@param {type:'string'}

#@markdown Folder in root containing training images
TRAIN_IMG = 'images' #@param {type:'string'}

#@markdown Folder in root containing training labels
TRAIN_LBL = 'labels' #@param {type:'string'}

#@markdown Path to test images
TEST_IMG = 'D:\\Images\\test' #@param {type:'string'}

#@markdown Save path for test results
SAVE_IMG = 'D:\\Images\\Results' #@param {type:'string'}

#@markdown Save path for new trained weights
SAVE_WGT = 'D:\\Weights\\Results' #@param {type:'string'}

#@markdown Name for new weight
WGT_NAME = 'unet_new_model.hdf5' #@param {type:'string'}

#@markdown Path to pre-trained weights
WEIGHT_PATH = 'D:\\Weights\\unet_noise_tissue.hdf5' #@param {type:'string'}

SAVE_WGT = os.path.join(SAVE_WGT,WGT_NAME)

#ROOT_PATH = os.path.dirname(os.path.commonprefix([TRAIN_IMG,TRAIN_LBL]))

#@markdown ### **B. Training or Testing?**
ToT = "Test" #@param ["Train","Test"] {type:"string"}

In [12]:
#@markdown ## **IV. Configuration**
#@markdown ### **A. Image and Train Parameters**

#@markdown ### Image file extension (e.g. png, tif, jpg)
ext = 'png' #@param ['png','tif','jpg']{type:'string'}

#@markdown ### Image color mode (grayscale, rgb)
COLOR = 'grayscale' #@param ['grayscale','rgb','rgba']{type:'string'}

#@markdown ### Select metric to monitor
METRIC = 'loss' #@param ['loss','val_loss','accuracy','val_accuracy']{type:'string'}

#@markdown ### Display train info every V epochs
VERBOSE = 2 #@param {type:"slider", min:0, max:100, step:1}

#@markdown ### Save best weight or last weight
SAVE_BEST = False #@param {type:"boolean"}

#@markdown ### Number of steps per epoch
STEPS =  100#@param {type: "integer"}

#@markdown ### Number of epochs
EPOCHS =  30#@param {type: "integer"}

#@markdown ### Size of Image (MxN)
M = 512 #@param {type: "integer"}
N = 512 #@param {type: "integer"}

#@markdown ___

#@markdown ### **B. Data Augmentation Parameters**
#@markdown Augmentations are randomly assigned

#@markdown ### Rotation Range (degrees)
ROT = 45 #@param {type:"slider", min:0, max:90, step:1}
ROT = ROT/100

#@markdown ### Width Shift Range (fraction of total image width)
WSR = 0.15 #@param {type:"slider", min:0, max:0.5, step:0.05}

#@markdown ### Horizontal Shift Range (fraction of total image height)
HSR = 0.15 #@param {type:"slider", min:0, max:0.5, step:0.05}

#@markdown ### Shear angle in counter-clockwise direction (degrees) 
SHEAR = 0 #@param {type:"slider", min:0, max:90, step:1}
SHEAR = SHEAR/100

#@markdown ### Zoom Scale Range
ZOOM = 0.15 #@param {type:"slider", min:0, max:0.50, step:0.05}

#@markdown ### Horizontal Flip
HF = True #@param {type:"boolean"}

#@markdown ### Vertical Flip
VF = True #@param {type:"boolean"}

#@markdown ### Fill mode
FM = 'nearest' #@param ["constant", "nearest", "reflect","wrap"]{type:"string"}


In [ ]:
os.chdir(UPATH)
#@markdown ___
#@markdown ## **V. Train Network**
#@markdown Using paths and parameters defined above.
from model import *
from data import *

data_gen_args = dict(rotation_range=ROT,
                    width_shift_range=WSR,
                    height_shift_range=HSR,
                    shear_range=SHEAR,
                    zoom_range=ZOOM,
                    horizontal_flip=HF,
                    vertical_flip=VF,
                    fill_mode=FM)

myGene = trainGenerator(1,ROOT_PATH, TRAIN_IMG,TRAIN_LBL,data_gen_args,save_to_dir = None, image_color_mode = COLOR)

model = unet()
model_checkpoint = ModelCheckpoint(SAVE_WGT, monitor=METRIC,verbose=VERBOSE, save_best_only=SAVE_BEST)
model.fit_generator(myGene,steps_per_epoch=STEPS,epochs=EPOCHS,callbacks=[model_checkpoint])

In [9]:
os.chdir(UPATH)
#@markdown ___
#@markdown ## **VI. Test Network**
#@markdown Test pretrained network
#@markdown
from model import *

model = load_model(WEIGHT_PATH)
test_path = TEST_IMG
save_path = SAVE_IMG
container = np.zeros((M,N,1,1))

def bin_ndarray(ndarray, new_shape, operation='sum'):
    """
    J.F. Sebastian
    Bins an ndarray in all axes based on the target shape, by summing or
        averaging.

    Number of output dimensions must match number of input dimensions and 
        new axes must divide old ones.

    """
    operation = operation.lower()
    if not operation in ['sum', 'mean']:
        raise ValueError("Operation not supported.")
    if ndarray.ndim != len(new_shape):
        raise ValueError("Shape mismatch: {} -> {}".format(ndarray.shape,
                                                           new_shape))
    compression_pairs = [(d, c//d) for d,c in zip(new_shape,
                                                  ndarray.shape)]
    flattened = [l for p in compression_pairs for l in p]
    ndarray = ndarray.reshape(flattened)
    for i in range(len(new_shape)):
        op = getattr(ndarray, operation)
        ndarray = op(-1*(i+1))
    return ndarray

def image_normalized(file_path):

    img = cv2.imread(file_path,0)
    img_shape = img.shape
    image_size = (img_shape[1],img_shape[0])
    img_standard = bin_ndarray(img*1.2, (M,N), operation='mean')
    #img_standard = cv2.resize(img, (512, 512), interpolation=cv2.INTER_CUBIC)
    img_new = img_standard
    img_new = np.asarray([img_new / 255.])
    return img_new,image_size

for name in os.listdir(test_path):
	image_path = os.path.join(test_path,name)
	if os.path.isdir(image_path):
		continue
	img,img_size = image_normalized(image_path)
	img = np.reshape(img,img.shape+(1,))
	results = model.predict(img)
	out = np.zeros(img.shape)
	#out = np.zeros((512,512));
	#print(results.shape)
	#out[results[0,:,:,0] > 0] = 1;
	out = 255*results[0,:,:,0];
	
	cv2.imwrite(os.path.join(save_path, ("%s") % (name)), out)

